In [1]:
#load in the generated story
with open("output/gpt-4o-mini/generated_story_level_2.txt", "r") as file:
    generated_story = file.read()

print(generated_story)

### Rapunzel

Once there was a man and a woman who wanted a baby. They wished for a long time. One day, the woman saw a garden from her window. It was full of beautiful flowers and herbs, but it had a high wall. The garden belonged to a powerful enchantress, and no one dared to go in.

The woman saw a plant called rampion in the garden. It looked so fresh and green that she really wanted to eat it. She felt so sick without it. Her husband noticed and asked, "What is wrong, dear wife?" 

"I want some rampion from the garden," she replied. "If I can't eat it, I will feel very sad!"

The husband cared for her and thought, "I will bring her some rampion." That night, he climbed over the wall and picked some. He gave it to his wife, and she made a salad. It tasted so good that she wanted more the next day.

The husband went back to the garden to get more rampion, but this time he saw the enchantress. She was angry. "How dare you steal my rampion?" she yelled. The husband was scared but expl

In [62]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel, ValidationError
import json

load_dotenv()

class CharacterImagePrompt(BaseModel):
    character_name: str
    character_pose: str
    character_clothing: str
    character_expression: str
    character_prompt: str

class StoryImagePrompts(BaseModel):
    background_image_prompt: str
    character_image_prompts: list[CharacterImagePrompt]

class Scene(BaseModel):
    scene_no: int
    story_text: str
    story_image_prompts: StoryImagePrompts

class StoryScenes(BaseModel):
    illustration_style: str
    scenes: list[Scene]

def split_story_into_scenes(story_text: str) -> StoryScenes:
    # Define a prompt to instruct the model to split the story into scenes
    prompt = f"Split the following story into scenes. Separate each scene with a new line. This is the story: {story_text}"

    # Call the OpenAI API
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": f"""You are an expert story illustrator creating a web app for children ages between 6-8 learning to read through stories. You are given a story text and before illustrating it, you need to 
                1. Read through the story text and chunk up the story into scenes. MAKE sure to have all the key scenes including the introduction, development, conflict, climax and resolution.
                2. Choose an illustration style for the story suitable for children ages between 6-8. Describe the style in detail so that can be used to generate consistent background images and characters.
                3. Make a list of all the characters that appear in the story. If the same character is referred to with different ways, eg. man, husband, Mike, etc. you should refer to this character using his actual name if it is mentioned in the story, else refer to this character using the name that is most commonly used in the story and which is easily understood by someone reading your output. ALWAYS maintain consistency when referring to the same character throughout your output.
                3. For each scene, describe the background scene in detail, including the
                    - setting, 
                    - time of day
                    - weather
                    - the mood.
                    DO NOT INCLUDE any characters in the background description.
                4. For each scene, describe the key characters, poses, clothing and expressions in detail so that it can be used to generate consistent images. The clothing description should be consistent for the same character unless it makes sense to change the character's clothing. Clothing description should include the type of clothes, composition, style, colour, texture, pattern, etc.              
                DO NOT alter the story text in any way.
                MAKE SURE TO use ALL the text provided in the story text."""
            },
            {
                "role": "user",
                "content": f"This is the story: {story_text}"
            }
        ],
        response_format=StoryScenes # Use "json" as the response format
    )

    # Parse the response into the StoryScenes model

    scenes = completion.choices[0].message.parsed
    return scenes
    
# Example usage
# scenes = split_story_into_scenes("Your story text here")
# print(scenes)

In [63]:
scenes = split_story_into_scenes(generated_story)
print(scenes)

illustration_style='The illustration style is whimsical and colorful, characterized by soft lines and rounded shapes that appeal to children ages 6-8. The color palette is vibrant, with bright primary colors accentuated by pastel shades, creating a cheerful atmosphere. The characters have exaggerated features, such as big eyes and expressive faces, to convey emotions easily. Backgrounds are rich in detail but not overly busy, featuring lush landscapes, enchanted forests, and charming castles, ensuring they support the storytelling without drawing attention away from the characters. The overall look is playful and inviting, encouraging a sense of wonder and imagination.' scenes=[Scene(scene_no=1, story_text='Once there was a man and a woman who wanted a baby. They wished for a long time. One day, the woman saw a garden from her window. It was full of beautiful flowers and herbs, but it had a high wall. The garden belonged to a powerful enchantress, and no one dared to go in.', story_ima

In [64]:
scenes_json = scenes.model_dump(mode="json")
print(json.dumps(scenes_json, indent=4))

{
    "illustration_style": "The illustration style is whimsical and colorful, characterized by soft lines and rounded shapes that appeal to children ages 6-8. The color palette is vibrant, with bright primary colors accentuated by pastel shades, creating a cheerful atmosphere. The characters have exaggerated features, such as big eyes and expressive faces, to convey emotions easily. Backgrounds are rich in detail but not overly busy, featuring lush landscapes, enchanted forests, and charming castles, ensuring they support the storytelling without drawing attention away from the characters. The overall look is playful and inviting, encouraging a sense of wonder and imagination.",
    "scenes": [
        {
            "scene_no": 1,
            "story_text": "Once there was a man and a woman who wanted a baby. They wished for a long time. One day, the woman saw a garden from her window. It was full of beautiful flowers and herbs, but it had a high wall. The garden belonged to a powerful

In [65]:
#write the output to file
with open("output/gpt-4o-mini/generated_story_scenes_level_2_clothing.json", "w") as file:
    json.dump(scenes_json, file, indent=4)